# 가게 리뷰 키워드 기반 코사인 유사도

## 전처리

In [1]:
import pandas as pd
import numpy as np
import json
import csv

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 파일 불러오기

'''csv_df = pd.read_csv('./keywordforpearsonr.csv', encoding='utf8')
csv_df'''

f = open('./keywordforpearsonr.csv', encoding='utf8')
rd = csv.reader(f)
csv_list = []
for l in rd:
    csv_list.append(l)
f.close()

csv_df = pd.DataFrame(csv_list)
csv_df

,0,1,2,3,4,5,6,7,8,9,10
0,526,안주,치킨,바지락,전통주,간술,결과,친구,모임,커플,웨이팅
1,5412,주류,미나리,막걸리,낯선,대낮,여자,멤버,예약,가능성,육회
2,&meal,바질,고구마,스프,샌드위치,시즌,파니니,단호박,크림,연어,청양
3,101번지남산돈까스 현대시티아울렛가산점,돈까스,남산,돈가스,쫄면,포장,와이프,김치우,모밀,부어,접객
4,1954무한등심 잠실점,등심,고기,불고기,무한리필,샐러드,선풍기,소고기,무한,할머니,언니
...,...,...,...,...,...,...,...,...,...,...,...
2065,히든브라운,하몽,임낫,브라운,식빵,요금,내야,메뉴판,자몽주스,플랫,왕복
2066,히카리 우동,우동,튀김,새우,시렁,분위기,유부초밥,맥주,커뮤니케이션,한텐,물수건
2067,히피스베이글,베이글,피스,스노,블루베리,비건,우이동,덕성여대,기적,초코,오레오
2068,힐사이드테이블,샌드위치,샐러드,단호박,두유,치즈,크림,흑임자,유치,치킨,세트


In [3]:
# 기존 인덱스(숫자) 버리고 상호명을 인덱스로 바꾸기

ls = csv_df.iloc[:,0]
ls = pd.Series(ls)

csv_df = csv_df.rename(index=ls)
csv_df.drop([0], inplace=True, axis=1)
csv_df

,1,2,3,4,5,6,7,8,9,10
526,안주,치킨,바지락,전통주,간술,결과,친구,모임,커플,웨이팅
5412,주류,미나리,막걸리,낯선,대낮,여자,멤버,예약,가능성,육회
&meal,바질,고구마,스프,샌드위치,시즌,파니니,단호박,크림,연어,청양
101번지남산돈까스 현대시티아울렛가산점,돈까스,남산,돈가스,쫄면,포장,와이프,김치우,모밀,부어,접객
1954무한등심 잠실점,등심,고기,불고기,무한리필,샐러드,선풍기,소고기,무한,할머니,언니
...,...,...,...,...,...,...,...,...,...,...
히든브라운,하몽,임낫,브라운,식빵,요금,내야,메뉴판,자몽주스,플랫,왕복
히카리 우동,우동,튀김,새우,시렁,분위기,유부초밥,맥주,커뮤니케이션,한텐,물수건
히피스베이글,베이글,피스,스노,블루베리,비건,우이동,덕성여대,기적,초코,오레오
힐사이드테이블,샌드위치,샐러드,단호박,두유,치즈,크림,흑임자,유치,치킨,세트


In [4]:
# csv -> dict

idx = list(ls)
contents = {}
for i in idx:
    lst = list(csv_df.loc[i])
    for j in range(10):
        if lst[j] == None:
            lst = lst[:j]
            break
    contents[i] = ' '.join(lst)

contents

{'526': '안주 치킨 바지락 전통주 간술 결과 친구 모임 커플 웨이팅',
 '5412': '주류 미나리 막걸리 낯선 대낮 여자 멤버 예약 가능성 육회',
 '&meal': '바질 고구마 스프 샌드위치 시즌 파니니 단호박 크림 연어 청양',
 '101번지남산돈까스 현대시티아울렛가산점': '돈까스 남산 돈가스 쫄면 포장 와이프 김치우 모밀 부어 접객',
 '1954무한등심 잠실점': '등심 고기 불고기 무한리필 샐러드 선풍기 소고기 무한 할머니 언니',
 '1994양과점': '케이크 맛집 크림 갸또 디저트 럼블 포장 핑크 바나나 에스프레소',
 '1인1상': '파스타 디저트 바나나 단점 주차장 엄마 마을 주차 그날 정해진',
 '33Acre': '분위기 카페 솔드아웃 대접 야외 테라스 사진 계절 특성 파리',
 '3대삼계장인': '삼계탕 포장 수비드 닭볶음탕 살밥 혼밥 꿀맛 고기 어르신 모시',
 '3일한우국밥': '국밥 신분 오스 고기 소자 곰탕 젓가락 갈비찜 갈비 덩이',
 '588제주흑도야지': '고기 고사리 볶음밥 신의한수 계란찜 사진 맛집 김치 목살 친구',
 '60년전통 신촌황소곱창 논현직영점': '한우 모듬 친구 특유 젓가락 절도 마법 경험 존맛 곱창',
 '6회말연어': '육회 연어 세트 소고기 존맛 초밥 얼그레이 무국 사시미 거짓말',
 '72420 노원점': '쌀국수 런치 안심 양지 볶음밥 세트 점심시간 추후 존중 한마디',
 '72420 성신여대점': '쌀국수 볶음밥 예약 짜조 고수 맛집 사이즈 날치 분위기 성신여대',
 '79파운야드 가산디지털단지점': '디저트 분위기 민트 카페 바닐라 지점 커피 크림 아침 요크',
 '8번가': '예약 안주 튀김 친구 호가 분위기 불하 해물짬뽕 짬뽕 차돌',
 '8빵': '레몬 케이크 식빵 쫀득함 채소 두유 감자 추정 시금치 토마토',
 '943 킹스크로스CAFE': '해리포터 사진 스무디 키위 카페 커피 친구 홍대 입장료 버터',
 '964커피&너만보여크루아상': '크루아상 녹차 딸기 디저트 카페 피

In [5]:
# dict -> dataframe

df = pd.DataFrame(contents, index = [0]).T
df

,0
526,안주 치킨 바지락 전통주 간술 결과 친구 모임 커플 웨이팅
5412,주류 미나리 막걸리 낯선 대낮 여자 멤버 예약 가능성 육회
&meal,바질 고구마 스프 샌드위치 시즌 파니니 단호박 크림 연어 청양
101번지남산돈까스 현대시티아울렛가산점,돈까스 남산 돈가스 쫄면 포장 와이프 김치우 모밀 부어 접객
1954무한등심 잠실점,등심 고기 불고기 무한리필 샐러드 선풍기 소고기 무한 할머니 언니
...,...
히든브라운,하몽 임낫 브라운 식빵 요금 내야 메뉴판 자몽주스 플랫 왕복
히카리 우동,우동 튀김 새우 시렁 분위기 유부초밥 맥주 커뮤니케이션 한텐 물수건
히피스베이글,베이글 피스 스노 블루베리 비건 우이동 덕성여대 기적 초코 오레오
힐사이드테이블,샌드위치 샐러드 단호박 두유 치즈 크림 흑임자 유치 치킨 세트


## TF-IDF

TF-IDF는 TF와 IDF를 곱한 값을 의미하는 데, 문서를 d, 단어를 t, 문서의 총 개수를 n이라고 표현할 때 다음과 같이 정의 가능하다.
- TF(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수
- DF(t) : 특정 단어 t가 등장한 문서의 수
- IDF(d,t) : df(t)에 반비례하는 수

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# df -> tf-idf vector

tfidf_vector = TfidfVectorizer()
tfidf_matrix = tfidf_vector.fit_transform(df[0]).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = ls)
tfidf_matrix

,가겐,가까이,가늠,가능성,가든,가라,가라아게,가락동,가래떡,가량,...,흥남,히든,히레,히레카츠,히로시마,히비스커스,히토메보레,힐링,힙해,힝구
0,,,,,,,,,,,,,,,,,,,,,
526,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5412,0.0,0.0,0.0,0.387539,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&meal,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101번지남산돈까스 현대시티아울렛가산점,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1954무한등심 잠실점,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
히든브라운,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
히카리 우동,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
히피스베이글,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# 유사도 계산을 위한 df

cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, index = ls, columns = ls)
cosine_sim_df

,526,5412,&meal,101번지남산돈까스 현대시티아울렛가산점,1954무한등심 잠실점,1994양과점,1인1상,33Acre,3대삼계장인,3일한우국밥,...,휴식,흥부골숯불돼지왕갈비,희게,희릿,히노야마,히든브라운,히카리 우동,히피스베이글,힐사이드테이블,힛더스팟 베이커리 에이트
0,,,,,,,,,,,,,,,,,,,,,
526,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.072319,0.045058
5412,0.000000,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.031076,0.000000,0.0,0.0,0.0,0.000000,0.000000
&meal,0.000000,0.0,1.000000,0.0,0.000000,0.038852,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.035103,0.000000,0.000000,0.0,0.0,0.0,0.220298,0.000000
101번지남산돈까스 현대시티아울렛가산점,0.000000,0.0,0.000000,1.0,0.000000,0.033588,0.000000,0.000000,0.030054,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
1954무한등심 잠실점,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.022002,0.022395,...,0.000000,0.024879,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.052567,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
히든브라운,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.000000
히카리 우동,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.019114,0.000000,0.000000,...,0.038155,0.000000,0.018683,0.020843,0.098117,0.0,1.0,0.0,0.000000,0.000000
히피스베이글,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.000000


In [11]:
# 추천 시스템 구현

def recommend(target_place, matrix = cosine_sim_df, items = df, k=10):
    recom_idx = matrix.loc[:, target_place].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_place = items.iloc[recom_idx, :].index
    target_title_list = np.full(len(range(k)), target_place)
    similarity = matrix[target_place].sort_values(ascending=False)[1:11]
    
    d = {
        'target_place': target_title_list,
        'recom_place' : recom_place,
        'similarity'  : similarity
    }
    return pd.DataFrame(d).reset_index(drop=True)

In [12]:
# 추천 예시

print(recommend('오츠커피 용산점'))

  target_place recom_place  similarity
0     오츠커피 용산점       얼터너티브    0.523911
1     오츠커피 용산점     카페버킷리스트    0.487427
2     오츠커피 용산점    아키비스트 서촌    0.435357
3     오츠커피 용산점        태양커피    0.424503
4     오츠커피 용산점      LOWIDE    0.385697
5     오츠커피 용산점        아우어룸    0.361087
6     오츠커피 용산점         도베르    0.342412
7     오츠커피 용산점        버키커피    0.306925
8     오츠커피 용산점        카페히든    0.298681
9     오츠커피 용산점       라운드앤드    0.288461
